In [19]:
# Libraries
import caveclient
import subprocess
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Quick notebook on how to create annotations of synapses given a bounding box from neuroglancer link
- Example: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/4993893092294656

### Helper functions and variables

In [7]:
new_annotation_template = { # standard spleuker annotation template for neuroglancer
      "type": "annotation",
      "source": {
        "url": "local://annotations",
        "transform": {
          "outputDimensions": {
            "x": [
              3.2e-8,
              "m"
            ],
            "y": [
              3.2e-8,
              "m"
            ],
            "z": [
              4.5e-8,
              "m"
            ]
          }
        }
      },
      "tab": "source",
      "annotations": [],
      "name": "annotation1"
}


def create_ng_link_with_annots(existing_json_state, new_annot_name, coordinates, client):
    """ 
    Given some annotation coordinates, and a new annotation layer, create a ng link (base using spelunker.cave) 
    """
    new_annots = create_copy_paste_annot_list(annot_pt_list = coordinates, mip=(32,32,45), connected=True)
    new_annotation_layer = new_annotation_template.copy()

    new_annotation_layer['name'] = new_annot_name
    new_annotation_layer['annotations'] = new_annots
    existing_json_state['layers'].append(new_annotation_layer)
    new_id = client.state.upload_state_json(existing_json_state)
    print('New neuroglancer id:', new_id)

    ng_link_url = 'https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/'
    print('Link generated')
    print(ng_link_url + str(new_id))
    return new_id, ng_link_url


def create_copy_paste_annot_list(annot_pt_list = [], mip=(32,32,45), description_list = None, connected=False, autocopy=False):
    """ 
    List of annotations to copy/paste to spleunker.cave 
    """

    if description_list == None:
        description_list = ["" for _ in range(len(annot_pt_list))]
    else:
        assert len(annot_pt_list) == len(description_list)

    annots = []
    for i, pt in enumerate(annot_pt_list):
        if connected:
            pt_dict = {
              "pointA": list([int(pt[0][0]/mip[0]), int(pt[0][1]/mip[1]), int(pt[0][2]/mip[2])]),
              "pointB": list([int(pt[1][0]/mip[0]), int(pt[1][1]/mip[1]), int(pt[1][2]/mip[2])]),
              "type": "line",
              "id": "b6f2611ba4basd3635da2ea158ee9853fab0dabf"+str(i), # random id. Needs to be unique for each annot
              "description": pt[2]
            }
        else:
            pt_dict = {"point": list([int(pt[0]/mip[0]), int(pt[1]/mip[1]), int(pt[2]/mip[2])]), 
                       "description": str(description_list[i]), 
                       "type": "point", "id": "b6f2611ba4basd3635da2ea158ee9853fab0dabf"+str(i)}
            
        annots.append(pt_dict)

    if autocopy:
        subprocess.run("pbcopy", text=True, input=json.dumps(annots))
    return annots

### Main run

In [8]:
# Setup caveclient
client = caveclient.CAVEclient("wclee_mouse_spinalcord_cltmr", )

In [15]:
# Get neuroglancer json state
ng_link_code = 6237693408182272 # last number of neuroglancer link
json_state = client.state.get_state_json(ng_link_code) # modify if needed

annots = json_state['layers'][4]['annotations'] # get annotation layer (modify if needed). Annot layer may not always be 4th/5th in line

In [26]:
# Create a coordinates list
coordinates = []

In [27]:
print("Collecting synapses:")

SYNAPSE_TABLE_NAME = 'synapses_v2'
SCALE_NG = (32,32,45) # resolution of the ng link

for annot in tqdm(annots):
    assert annot['type'] == 'axis_aligned_bounding_box'

    # Convert to real-world nm coordinates
    minb = [min(annot['pointA'][0]*SCALE_NG[0], annot['pointB'][0]*SCALE_NG[0]), 
            min(annot['pointA'][1]*SCALE_NG[1], annot['pointB'][1]*SCALE_NG[1],), 
            min(annot['pointA'][2]*SCALE_NG[2], annot['pointB'][2]*SCALE_NG[2])]
    
    maxb = [max(annot['pointA'][0]*SCALE_NG[0], annot['pointB'][0]*SCALE_NG[0]), 
            max(annot['pointA'][1]*SCALE_NG[1], annot['pointB'][1]*SCALE_NG[1],), 
            max(annot['pointA'][2]*SCALE_NG[2], annot['pointB'][2]*SCALE_NG[2])]

    bounding_box = [minb, maxb]
    synapses_df = client.materialize.query_table(SYNAPSE_TABLE_NAME, filter_spatial_dict  = {'ctr_pt_position': bounding_box})

    for i, row in synapses_df.iterrows():
        coordinates.append([row['pre_pt_position'], row['post_pt_position'], str(row['id'])])  

print('TOTAL number of synapses:', len(coordinates))

 11%|██████████▊                                                                                      | 1/9 [01:33<12:29, 93.65s/it]

Number of synapses: 110
TOTAL number of synapses: 110


 22%|█████████████████████▌                                                                           | 2/9 [02:21<07:47, 66.85s/it]

Number of synapses: 148
TOTAL number of synapses: 258


 33%|████████████████████████████████▎                                                                | 3/9 [02:28<03:56, 39.45s/it]

...

Number of synapses: 104
TOTAL number of synapses: 632


 89%|██████████████████████████████████████████████████████████████████████████████████████▏          | 8/9 [03:01<00:10, 10.53s/it]

Number of synapses: 47
TOTAL number of synapses: 679


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [03:08<00:00, 20.94s/it]

Number of synapses: 87
TOTAL number of synapses: 766


In [35]:
# Create ng link with synapses
create_ng_link_with_annots(json_state, new_annot_name='auto synapses', coordinates=coordinates, client=client)